# Data ETL Pipeline

In [1]:
import tensorflow.keras as keras
(xtrain, ytrain), (xtest, ytest) = keras.datasets.fashion_mnist.load_data()


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
print("xtrain Shape :", xtrain.shape)
print("ytrain Shape :", ytrain.shape)
print("xtest Shape :", xtest.shape)
print("ytest Shape :", ytest.shape)

xtrain Shape : (60000, 28, 28)
ytrain Shape : (60000,)
xtest Shape : (10000, 28, 28)
ytest Shape : (10000,)


## clean and transform the data

* we will normalize the pixel values to be between 0 and 1 and reshape the data into a 4D tensor:

In [4]:
import numpy as np

xtrain = xtrain.astype('float32') / 255
xtest = xtest.astype('float32') / 255

xtrain = np.reshape(xtrain, (xtrain.shape[0], 28, 28, 1))
xtest = np.reshape(xtest, (xtest.shape[0], 28, 28, 1))

In [5]:
print("xtrain Shape :", xtrain.shape)
print("ytrain Shape :", ytrain.shape)
print("xtest Shape :", xtest.shape)
print("ytest Shape :", ytest.shape)

xtrain Shape : (60000, 28, 28, 1)
ytrain Shape : (60000,)
xtest Shape : (10000, 28, 28, 1)
ytest Shape : (10000,)


## loading the data into a database
* We will use SQLite to create a database and load the data into it

In [6]:
import sqlite3

conn = sqlite3.connect('fashion_mnist.db')

conn.execute('''CREATE TABLE IF NOT EXISTS images
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
             image BLOB NOT NULL,
             label INTEGER NOT NULL);''')

for i in range(xtrain.shape[0]):
    conn.execute('INSERT INTO images (image, label) VALUES (?, ?)',
                [sqlite3.Binary(xtrain[i]), ytrain[i]])

conn.commit()

for i in range(xtest.shape[0]):
    conn.execute('INSERT INTO images (image, label) VALUES (?, ?)',
                [sqlite3.Binary(xtest[i]), ytest[i]])

conn.commit()

conn.close()

#### In the above code:

* **Import sqlite3 Library:** The first line imports the sqlite3 library, which allows us to interact with SQLite databases in Python.

* **Establish Database Connection:** We create a connection to the SQLite database.

* **Create a Table:** We define a new table named “images” in the database to store image data and labels.

* **Insert Training Data:** We loop through the training data, adding each image and its label into the “images” table.

* **Save Changes:** We use the commit() method to save the changes made to the database.

* **Insert Test Data:** We loop through the test data, adding each image and its label into the “images” table.

* **Save Changes Again:** We call commit() once more to save the changes made during the test data insertion.

* **Close Database Connection:** Finally, we close the connection to the SQLite database to complete the operations.

This is the process of building a Data ETL pipeline with Python. Our ETL pipeline extracts the Fashion MNIST dataset, transforms it as needed, and loads it into an SQLite database, allowing for convenient future access and data manipulation.

## reading the data stored on the SQLite database

In [9]:
conn = sqlite3.connect('fashion_mnist.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM images')
rows = cursor.fetchall()

data = pd.read_sql_query('SELECT * FROM images', conn)

# Summary

The Data ETL process involves extracting data from a source, transforming it through various processes, and then loading it into a database. ETL stands for Extract, Transform, and Load, which are the key stages of this data management approach.